In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import operator 
from pmdarima.arima import auto_arima
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
from datetime import date
import operator
from matplotlib import pyplot

In [2]:
confirmed_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
confirmed_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,367,423,444,484,521,555,607,665,714,784
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,377,383,400,409,416,433,446,467,475,494
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1423,1468,1572,1666,1761,1825,1914,1983,2070,2160
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,525,545,564,583,601,601,638,646,659,673
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,16,17,19,19,19,19,19,19,19,19


In [3]:
deaths_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
deaths_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,11,14,14,15,15,18,18,21,23,25
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,21,22,22,23,23,23,23,23,24,25
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,173,193,205,235,256,275,293,313,326,336
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,21,22,23,25,26,26,29,29,31,33
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [4]:
recoveries_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
recoveries_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,18,18,29,32,32,32,32,32,40,43
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,116,131,154,165,182,197,217,232,248,251
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,90,113,237,347,405,460,591,601,691,708
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,31,39,52,58,71,71,128,128,128,169
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,4,4,4,5,5


In [5]:
colnames = confirmed_cases.columns.values
colnames

array(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20',
       '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20',
       '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20',
       '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20',
       '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20',
       '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20',
       '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20',
       '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20',
       '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20',
       '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20',
       '4

In [6]:
confirmed = confirmed_cases.loc[:, colnames[4]:colnames[-1]]
deaths = deaths_cases.loc[:, colnames[4]:colnames[-1]]
recoveries = recoveries_cases.loc[:, colnames[4]:colnames[-1]]

In [7]:
time_date = confirmed.columns.values

In [8]:
all_days = np.array([i for i in range(len(time_date))]).reshape(-1, 1)

# Predictions for days ahead

In [9]:
#pred_days_ahead = 2
#pred_days_ahead = 7
pred_days_ahead = 30
predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

predictions_ahead_plus_day = np.array([i for i in range(len(time_date)+pred_days_ahead+1)]).reshape(-1, 1)

In [10]:
start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
predictions_ahead_dates = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day))]

In [11]:
#countries2 = ['Greece', 'Italy', 'Spain', 'Turkey','Cyprus', 'Switzerland']
#countries= confirmed_cases.iloc[35:46,1].tolist()#keep all  country names in a list
#countries=countries+countries2
#province=confirmed_cases.iloc[35:46,0].tolist()#province list with nan
#prov=[float("NaN")]*6
#province=province+prov
countries1=confirmed_cases['Country/Region'].tolist()
countries= [x for x in countries1 if x != 'Canada']
#province=confirmed_cases['Province/State'].tolist()
#countries = ['Algeria','France', 'France','Greece']
#province=[float("NaN"),'Martinique', float("NaN"),float("NaN")]
#province
#'French Guiana','French Polynesia','Guadeloupe','Mayotte','New Caledonia','Reunion','Saint Barthelemy',
       #'St Martin','
#countries=['Benin']
#province=[float("NaN")]
province=[confirmed_cases['Province/State'][x] for x in range(len(confirmed_cases['Province/State'])) if confirmed_cases['Country/Region'][x] != 'Canada' ]

In [12]:
#'Alberta' in province

In [13]:
#len(countries)== len(province)

In [14]:
prediction_models = dict()
prediction_models_confirmed = {'SVR': 0}
prediction_models_deaths = {'SVR': 0}
prediction_models_recoveries = {'SVR': 0}


In [15]:
# Prediction Models

from sklearn import model_selection
from sklearn.svm import SVR
def SVM_regression(train_x, train_y, test_x, test_y, X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
   # c = [0.01, 0.1, 1]
   # gamma = [0.01, 0.1, 1]
   #epsilon = [0.01, 0.1, 1]
   #shrinking = [True, False]
    #degree = [3, 4, 5, 6]
  #grid_parameters = {'C': c, 'gamma' : gamma ,'epsilon': epsilon, 'shrinking' : shrinking, 'degree':degree} 
   #regressor = RandomizedSearchCV(SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1, C=0.1), grid_parameters, scoring='neg_mean_squared_error', 
    #                               cv=5)
    
    if (subpopulation == 'confirmed'):
         degree = 3
    elif (subpopulation == 'deaths'):
        degree = 4
    else :
        degree = 5


    regressor = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=degree, C=0.1)
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['SVR'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['SVR'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['SVR'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions) 
    

In [16]:
from sklearn.ensemble import RandomForestRegressor
def RF_regression(train_x, train_y, test_x, test_y,X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    grid_parameters = {'n_estimators': n_estimators}
#    regressor = RandomizedSearchCV(RandomForestRegressor(), grid_parameters, scoring='neg_mean_squared_error', 
#                                   cv=3)
    
    regressor = RandomForestRegressor(n_estimators = 200, random_state = 42) 
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)


In [17]:
# Gradient Boosted Regression Trees (GBRT) 
from sklearn.ensemble import GradientBoostingRegressor

def Grad_Boost_regression(train_x, train_y, test_x, test_y,X_test,subpopulation, X_train, Y_train):
   
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    loss = ['ls','lad','huber']
    grid_parameters = {'n_estimators': n_estimators}
  #  regressor = RandomizedSearchCV(GradientBoostingRegressor(learning_rate=0.1), grid_parameters, scoring='neg_mean_squared_error', 
 #                                  cv=3)
    
    regressor =  GradientBoostingRegressor(n_estimators=500, learning_rate=0.1,
                                            max_depth=1, random_state=0, loss='huber')
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)



In [18]:
#check if a timeseries is stationary
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()     
    rolstd = timeseries.rolling(12).std()

    #Plot rolling statistics:
    #orig = plt.plot(timeseries, color='blue',label='Original')
    #mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    #std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    #plt.legend(loc='best')
    #plt.title('Rolling Mean & Standard Deviation')
    #plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    #print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput
    

In [19]:
def pred_with_arima(timeseries,n_days_ahead):
    
    #find the optimal d 
    diff=timeseries-timeseries.shift(1)
    for d in range(1,10):
        if (test_stationarity(diff.dropna())['p-value']<0.1):
            #print (d) 
            break
        else: 
            d=d+1 
            diff=diff-diff.shift(1)
    #use the arima model to make predictions, use original timeseries with optimal d:
    
    model = pm.auto_arima(timeseries, start_p=0, start_q=0,
                             max_p=4, max_q=4,
                              seasonal=False,
                             d=d, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=False)
    print(model)
    forecasts, conf_int =model.predict(n_days_ahead,return_conf_int=True ,alpha=0.05)
    return forecasts , conf_int # returns a tuple, array of forecasts and all conf int

In [20]:
def Arima_Analysis(region,pred_days_ahead):
   #create a dataframe for timeseries analysis
    date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
    df = pd.DataFrame(date_rng, columns=['date'])
    df['cases']=np.transpose(region) #this is a dataframe with dates and cases.
    #df['datetime'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    #df.drop(['date'], axis=1, inplace=True)
    #df.plot( style='k.')
    #pyplot.show()
    df=df[df['cases'] != 0] #remove zeroes, they do not help for analysis. 
    if (len(df)>1):
        if(df['cases'][-1]-df['cases'][0]>5):
            pred_ts,conf_int=pred_with_arima(df['cases'],pred_days_ahead)  #.tolist() #this is an array converted to a list
        else:
            pred_ts=[df['cases'][-1]+1]*pred_days_ahead
            conf_int=[np.array([df['cases'][-1],df['cases'][-1]+2])]*pred_days_ahead
    else:
        pred_ts=np.zeros(pred_days_ahead)
        conf_int=[np.zeros(pred_days_ahead)] * pred_days_ahead
    return pred_ts, conf_int

In [21]:
# region = confirmed_cases.loc[(confirmed_cases['Country/Region'] == 'Canada') & (confirmed_cases['Province/State'] == 'Alberta')] 
# region =  np.array(region.loc[:, colnames[4]:colnames[-1]])   
# date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
# df = pd.DataFrame(date_rng, columns=['date'])
# df['cases']=np.transpose(region)
# df = df.set_index('date')
# df=df[df['cases'] != 0]

In [22]:
#prediction, confidence= Arima_Analysis(region,2)

In [23]:
#pred_with_arima(df['cases'],2)[0]
#prediction

In [24]:
import operator
def subpopulation_Analysis(cases,country,province, all_days,predictions_ahead,subpopulation):
    
    pred  = list()
    if pd.isnull(province):
        region = cases.loc[(cases['Country/Region'] == country) & (pd.isnull(cases['Province/State']))]
    else :
        region = cases.loc[(cases['Country/Region'] == country) & (cases['Province/State'] == province)]
    print(region)
    
    reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
    region =  np.array(region.loc[:, colnames[4]:colnames[-1]]).reshape(-1, 1)
    
    
    #train_x, test_x, train_y, test_y = model_selection.train_test_split(all_days,region,test_size = 0.04,random_state = 42,shuffle=False)
    
    #pred_svm =  SVM_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_svm)
    
    pred_arima, conf_int=Arima_Analysis(reg,pred_days_ahead)
    #pred.append(pred_svm)
    #pred_RF = RF_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_RF)

    
   # pred_gb =  Grad_Boost_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
   # pred.append(pred_gb)


    if (subpopulation == 'confirmed'): 
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_confirmed.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    elif (subpopulation == 'deaths'):
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_deaths.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    else :
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_recoveries.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
        
    

    best_pred_ahead = best_pred[-1]
    best_conf_int=conf_int[-1]
    return best_pred_ahead, region, best_pred, best_conf_int

In [25]:
#pred_file = str(2)+"day_prediction_" +  date.today().strftime('%Y-%m-%d') + ".csv"
#pred_file = str(7)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file = str(30)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file

'30day_prediction_2020-04-16.csv'

In [26]:
columns = ['Province/State','Country','Target/Date','N','low95N','high95N',
          'R','low95R','high95R','D','low95D','high95D','T','low95T','high95T',
           'M','low95M','high95M','C','low95C','high95C']

            

k = 0
for co in range(len(countries)):
    
    total_predictions = pd.DataFrame(columns = columns)
    k = k+1
   
    best_pred_confirmed, region_con, best_pred_con, conf_int_con = subpopulation_Analysis(confirmed_cases,countries[co],province[co],
                                                 all_days,predictions_ahead,"confirmed")
  
    best_pred_deaths, region_d, best_pred_d, conf_int_d = subpopulation_Analysis(deaths_cases,countries[co],province[co],
                                              all_days,predictions_ahead,"deaths")
    
    best_pred_recoveries, region_r, best_pred_r, conf_int_r = subpopulation_Analysis(recoveries_cases,countries[co],province[co],
                                                  all_days,predictions_ahead,"recoveries")
    
    # Mortality rate
    if (conf_int_con[0]==0):
        mort_pred=0
        mort_pred_low=0
        mort_pred_up=0
    else:
        mort_pred  = best_pred_deaths/best_pred_confirmed
        mort_pred_low= conf_int_d[0]/conf_int_con[1]
        mort_pred_up=conf_int_d[1]/conf_int_con[0]
    
    #fix province
    province_name=''
    if (pd.notnull(province[co])) :
        province_name=province[co]
    
    total_predictions = total_predictions.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed) ,
                                                'low95N':int(conf_int_con[0]),
                                                'high95N': int(conf_int_con[1]),
                                                'R': int(best_pred_recoveries) ,
                                                'low95R' : int(conf_int_r[0]),
                                                'high95R' : int(conf_int_r[1]),
                                                'D': int(best_pred_deaths) ,
                                                'low95D' : int(conf_int_d[0]),
                                                'high95D': int(conf_int_d[1]),
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred,
                                                'low95M': mort_pred_low,
                                                'high95M': mort_pred_up ,
                                                'C': '' ,
                                                'low95C': '' ,
                                                'high95C': '' },ignore_index = True)
    
    if (k==1):
        header = True
    else:
        header = False
    
    total_predictions.to_csv(pred_file, sep=',',index = False,  mode='a', header = header)
    

  Province/State Country/Region   Lat  Long  1/22/20  1/23/20  1/24/20  \
0            NaN    Afghanistan  33.0  65.0        0        0        0   

   1/25/20  1/26/20  1/27/20  ...  4/6/20  4/7/20  4/8/20  4/9/20  4/10/20  \
0        0        0        0  ...     367     423     444     484      521   

   4/11/20  4/12/20  4/13/20  4/14/20  4/15/20  
0      555      607      665      714      784  

[1 rows x 89 columns]
Fit ARIMA: (0, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=489.120, BIC=492.903, Time=0.017 seconds
Fit ARIMA: (0, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=436.419, BIC=442.094, Time=0.152 seconds
Fit ARIMA: (0, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=397.379, BIC=404.946, Time=0.422 seconds
Fit ARIMA: (0, 3, 3)x(0, 0, 0, 0) (constant=True); AIC=393.995, BIC=403.454, Time=0.469 seconds
Fit ARIMA: (0, 3, 4)x(0, 0, 0, 0) (constant=True); AIC=394.930, BIC=406.281, Time=0.438 seconds
Fit ARIMA: (1, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=450.797, BIC=456.472, Time=0.041 secon

Fit ARIMA: (0, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=249.678, BIC=259.344, Time=0.213 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=244.577, BIC=249.410, Time=0.062 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=245.823, BIC=252.267, Time=0.091 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=247.556, BIC=255.611, Time=0.195 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=249.070, BIC=258.735, Time=0.263 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=251.018, BIC=262.294, Time=0.303 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=245.695, BIC=252.139, Time=0.085 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=247.694, BIC=255.748, Time=0.147 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=249.710, BIC=259.376, Time=0.291 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=250.974, BIC=262.250, Time=0.447 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (const

Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=457.659, BIC=467.118, Time=0.338 seconds
Fit ARIMA: (2, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=459.184, BIC=470.535, Time=0.420 seconds
Fit ARIMA: (2, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=459.226, BIC=472.469, Time=0.659 seconds
Fit ARIMA: (3, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=458.384, BIC=467.843, Time=0.231 seconds
Fit ARIMA: (3, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=459.675, BIC=471.026, Time=0.308 seconds
Fit ARIMA: (3, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=458.068, BIC=471.311, Time=0.558 seconds
Fit ARIMA: (4, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=458.161, BIC=469.512, Time=0.213 seconds
Fit ARIMA: (4, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=458.072, BIC=471.315, Time=0.528 seconds
Total fit time: 6.039 seconds
ARIMA(maxiter=50, method='lbfgs', order=(0, 2, 4), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,

Fit ARIMA: (4, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=338.686, BIC=351.015, Time=0.409 seconds
Total fit time: 3.885 seconds
ARIMA(maxiter=50, method='lbfgs', order=(2, 2, 0), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)
  Province/State Country/Region      Lat    Long  1/22/20  1/23/20  1/24/20  \
3            NaN        Andorra  42.5063  1.5218        0        0        0   

   1/25/20  1/26/20  1/27/20  ...  4/6/20  4/7/20  4/8/20  4/9/20  4/10/20  \
3        0        0        0  ...      21      22      23      25       26   

   4/11/20  4/12/20  4/13/20  4/14/20  4/15/20  
3       26       29       29       31       33  

[1 rows x 89 columns]
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=78.508, BIC=80.864, Time=0.016 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=80.507, BIC=84.041, Time=0.039 seconds
Fit ARIMA: (0, 1, 2

Fit ARIMA: (0, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=125.296, BIC=131.282, Time=0.057 seconds
Fit ARIMA: (0, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=126.886, BIC=134.368, Time=0.109 seconds
Fit ARIMA: (0, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=128.062, BIC=137.041, Time=0.136 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=124.084, BIC=128.574, Time=0.041 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=125.587, BIC=131.573, Time=0.109 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=127.210, BIC=134.693, Time=0.127 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=125.357, BIC=134.336, Time=0.178 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=126.155, BIC=136.630, Time=0.480 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=125.148, BIC=131.134, Time=0.031 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=127.003, BIC=134.486, Time=0.096 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (const

...

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\statsmodels\regression\linear_model.py:889: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


Fit ARIMA: (0, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=74.013, BIC=76.684, Time=0.036 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=75.014, BIC=78.575, Time=0.363 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=75.932, BIC=79.493, Time=0.051 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=76.969, BIC=81.421, Time=0.304 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=77.831, BIC=82.282, Time=0.065 seconds
Fit ARIMA: (3,

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=40.692, BIC=43.524, Time=0.253 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=42.129, BIC=45.669, Time=0.452 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=44.094, BIC=48.342, Time=0.549 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=42.861, BIC=45.694, Time=0.061 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=41.934, BIC=45.474, Time=0.342 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=43.904, BIC=48.153, Time=0.486 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=44.199, BIC=47.739, Time=0.098 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=43.931, BIC=48.179, Time=0.668 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=46.252, BIC=51.208, T

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=49.670, BIC=51.261, Time=0.257 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=51.236, BIC=53.225, Time=0.275 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=51.894, BIC=54.680, Time=0.492 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=51.207, BIC=52.799, Time=0.058 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=50.323, BIC=52.313, Time=0.252 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=50.485, BIC=52.872, Time=0.408 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=nan, BIC=nan, Time=nan seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=52.674, BIC=54.663, Time=0.107 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=51.908, BIC=54.295, Time=0.561 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=53.685, BIC=56.470, T

    Province/State Country/Region        Lat       Long  1/22/20  1/23/20  \
249            NaN          Yemen  15.552727  48.516388        0        0   

     1/24/20  1/25/20  1/26/20  1/27/20  ...  4/6/20  4/7/20  4/8/20  4/9/20  \
249        0        0        0        0  ...       0       0       0       0   

     4/10/20  4/11/20  4/12/20  4/13/20  4/14/20  4/15/20  
249        0        0        0        0        0        0  

[1 rows x 89 columns]
